# Varational Autoencoders in ZhuSuan

### This code implements a VAE with Gaussian prior and Bernoulli likelihood based on ZhuSuan.
#### The framework has been setup, please only fill the space with "TODO" comments.
#### You may see some detailed instructions in the comments. For detailed usage of ZhuSuan, please see the documentation on http://zhusuan.readthedocs.io/en/latest/concepts.html

#### If you have any questions, please contact with me: lucheng.lc15@gmail.com

In [1]:
import os
import time
import tensorflow as tf
import numpy as np
import zhusuan as zs

import matplotlib.pyplot as plt
from skimage import io, img_as_ubyte
from skimage.exposure import rescale_intensity

## I. Load MNIST Dataset
We use tensorflow tools to load dataset for convenience.

In [2]:
(data_train, label_train), _ = tf.keras.datasets.mnist.load_data()

In [3]:
print(data_train.shape, label_train.shape)

(60000, 28, 28) (60000,)


## II. Some Utils for Training and Visualization

Here are some functions for next steps. You don't need to know the details.

In [4]:
def iterbatches(arrays, *, num_batches=None, batch_size=None, shuffle=True, include_final_partial_batch=True):
    assert (num_batches is None) != (batch_size is None), 'Provide num_batches or batch_size, but not both'
    n = arrays[0].shape[0]
    assert all(a.shape[0] == n for a in arrays[1:])
    inds = np.arange(n)
    if shuffle: np.random.shuffle(inds)
    sections = np.arange(0, n, batch_size)[1:] if num_batches is None else num_batches
    for batch_inds in np.array_split(inds, sections):
        if include_final_partial_batch or len(batch_inds) == batch_size:
            yield tuple(a[batch_inds] for a in arrays)

In [5]:
def save_image_collections(x, filename, shape=(10, 10), scale_each=False,
                           transpose=False):
    """
    :param shape: tuple
        The shape of final big images.
    :param x: numpy array
        Input image collections. (number_of_images, rows, columns, channels) or
        (number_of_images, channels, rows, columns)
    :param scale_each: bool
        If true, rescale intensity for each image.
    :param transpose: bool
        If true, transpose x to (number_of_images, rows, columns, channels),
        i.e., put channels behind.
    :return: `uint8` numpy array
        The output image.
    """
    n = x.shape[0]
    if transpose:
        x = x.transpose(0, 2, 3, 1)
    if scale_each is True:
        for i in range(n):
            x[i] = rescale_intensity(x[i], out_range=(0, 1))
    n_channels = x.shape[3]
    x = img_as_ubyte(x)
    r, c = shape
    if r * c < n:
        print('Shape too small to contain all images')
    h, w = x.shape[1:3]
    ret = np.zeros((h * r, w * c, n_channels), dtype='uint8')
    for i in range(r):
        for j in range(c):
            if i * c + j < n:
                ret[i * h:(i + 1) * h, j * w:(j + 1) * w, :] = x[i * c + j]
    ret = ret.squeeze()
    io.imsave(filename, ret)

## III. Build the Generative Model by ZhuSuan

Define the generative model according to the generative process.

* TODO: complete the network.
    

In [6]:
@zs.meta_bayesian_net(scope="gen", reuse_variables=True)
def build_gen(x_dim, z_dim, n, n_particles=1):
    bn = zs.BayesianNet()
    z_mean = tf.zeros([n, z_dim])
    z = bn.normal("z", z_mean, std=1., group_ndims=1, n_samples=n_particles)
    h = tf.layers.dense(z, 500, activation=tf.nn.relu)
    ''' 
        TODO: add one more mlp layer of 500 hidden units with h as the input and return h
    '''
    h_1 = tf.layers.dense(h, 500, activation=tf.nn.relu)
    
    x_logits = tf.layers.dense(h_1, x_dim)
    bn.deterministic("x_mean", tf.sigmoid(x_logits))
    bn.bernoulli("x", x_logits, group_ndims=1)
    return bn

Define the variational posterior model.
- TODO: complete the network

In [7]:
@zs.reuse_variables(scope="q_net")
def build_q_net(x, z_dim, n_z_per_x):
        '''
        TODO: define a Bayesian network
        HINT: see the generative model
        '''
    bn = zs.BayesianNet()
    x = tf.cast(x, tf.float32)
            '''
            TODO: add two more mlp layers of 500 hidden units
            HINT: from x to h
            '''
        
    x_1 = tf.layers.dense(x, 500, activation=tf.nn.relu)
    h = tf.layers.dense(x_1, 500, activation=tf.nn.relu)
        
    z_mean = tf.layers.dense(h, z_dim)
    z_logstd = tf.layers.dense(h, z_dim)
        '''
            TODO: define q(z|x) using the Gaussian distribution of ZhuSuan
                > given input
                    - "z", z_mean, z_logstd (note that it is not std), n_z_per_x
                    - set group_ndims as 1
                > e.g.
                    - z = bn.normal("z", ..., logstd=..., group_ndims=..., n_samples=...)
        '''
        
    bn.normal("z", z_mean, logstd=z_logstd, n_samples = n_z_per_x, group_ndims=1)
    return bn

W0703 08:31:26.892318 140313118631744 module_wrapper.py:139] From /usr/local/lib/python3.6/dist-packages/zhusuan/framework/utils.py:106: The name tf.make_template is deprecated. Please use tf.compat.v1.make_template instead.



## IV. Set the Hyperparameters

The following hyperparameters work well. However, you can modify them anyway.

In [8]:
x_dim = 28 * 28
z_dim = 40
n_particles = 1

learning_rate = 0.001
epochs = 5000
batch_size = 128
save_freq = 50

## V. Build the Model and the Loss

In [9]:
x_input = tf.placeholder(tf.float32, shape=[None, x_dim], name="x")
# Change x to binary form.
x = tf.cast(tf.less(tf.random_uniform(tf.shape(x_input)), x_input), tf.int32)
n = tf.placeholder(tf.int32, shape=[], name="n")

model = build_gen(x_dim, z_dim, n, n_particles)
variational = build_q_net(x, z_dim, n_particles)

# Define the ELBO and optimize it by SGVB.
lower_bound = zs.variational.elbo(model, {"x": x}, variational=variational, axis=0)
cost = tf.reduce_mean(lower_bound.sgvb())
lower_bound = tf.reduce_mean(lower_bound)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
infer_op = optimizer.minimize(cost)

x_gen = tf.reshape(model.observe()["x_mean"], [-1, 28, 28, 1])

W0703 08:33:26.479383 140313118631744 deprecation.py:323] From <ipython-input-7-c754508f8819>:14: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.Dense instead.
W0703 08:33:26.482660 140313118631744 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/layers/core.py:187: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
W0703 08:33:26.542929 140313118631744 module_wrapper.py:139] From /usr/local/lib/python3.6/dist-packages/zhusuan/distributions/univariate.py:167: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W0703 08:33:26.560875 140313118631744 module_wrapper.py:139] From /usr/local/lib/python3.6/dist-packages/zhusuan/distributions/univariate.py:100: The name tf.log is deprecated. Please use 

## VI. Train the Model and Save the Generated Samples

In [10]:
# All generated samples are saved to this path.
result_path = 'images'
if not os.path.exists(result_path):
    os.mkdir(result_path)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(1, epochs + 1):
        time_epoch = -time.time()
        lbs = []
        for i_step, (x_batch, _) in enumerate(iterbatches(
            [data_train, label_train], batch_size=batch_size, include_final_partial_batch=False,
        )):
            x_batch = x_batch.reshape([-1, 28*28])
            _, lb = sess.run([infer_op, lower_bound],
                             feed_dict={x_input: x_batch,
                                        n: batch_size})
            lbs.append(lb)
        time_epoch += time.time()
        print("Epoch {} ({:.1f}s): Lower bound = {}".format(
            epoch, time_epoch, np.mean(lbs)))

        if epoch % save_freq == 0 or epoch == 1:
            images = sess.run(x_gen, feed_dict={n: 100})
            name = os.path.join(result_path,
                                "vae_epoch_{}.png".format(epoch))
            save_image_collections(images, name)

Epoch 1 (2.2s): Lower bound = -174.61705017089844


/usr/local/lib/python3.6/dist-packages/skimage/util/dtype.py:135: UserWarning: Possible precision loss when converting from float32 to uint8
  .format(dtypeobj_in, dtypeobj_out))


Epoch 2 (1.7s): Lower bound = -113.34323120117188
Epoch 3 (1.7s): Lower bound = -100.11080932617188
Epoch 4 (1.8s): Lower bound = -94.73204040527344
Epoch 5 (1.7s): Lower bound = -91.5213851928711
Epoch 6 (1.7s): Lower bound = -89.13658142089844
Epoch 7 (1.7s): Lower bound = -87.37793731689453
Epoch 8 (1.7s): Lower bound = -85.88752746582031
Epoch 9 (1.7s): Lower bound = -84.78380584716797
Epoch 10 (1.7s): Lower bound = -83.8565444946289
Epoch 11 (1.8s): Lower bound = -83.07714080810547
Epoch 12 (1.8s): Lower bound = -82.37248992919922
Epoch 13 (1.7s): Lower bound = -81.7703628540039
Epoch 14 (1.7s): Lower bound = -81.17167663574219
Epoch 15 (1.8s): Lower bound = -80.72090148925781
Epoch 16 (1.8s): Lower bound = -80.2710952758789
Epoch 17 (1.8s): Lower bound = -79.78621673583984
Epoch 18 (1.7s): Lower bound = -79.38304138183594
Epoch 19 (1.7s): Lower bound = -79.07406616210938
Epoch 20 (1.7s): Lower bound = -78.73430633544922
Epoch 21 (1.7s): Lower bound = -78.42970275878906
Epoch 22 (

Epoch 166 (1.7s): Lower bound = -69.4188232421875
Epoch 167 (1.7s): Lower bound = -69.41565704345703
Epoch 168 (1.7s): Lower bound = -69.43385314941406
Epoch 169 (1.7s): Lower bound = -69.46231842041016
Epoch 170 (1.7s): Lower bound = -69.38480377197266
Epoch 171 (1.7s): Lower bound = -69.37969207763672
Epoch 172 (1.7s): Lower bound = -69.38916778564453
Epoch 173 (1.8s): Lower bound = -69.33961486816406
Epoch 174 (1.7s): Lower bound = -69.37039947509766
Epoch 175 (1.7s): Lower bound = -69.33480834960938
Epoch 176 (1.7s): Lower bound = -69.26542663574219
Epoch 177 (1.7s): Lower bound = -69.34695434570312
Epoch 178 (1.7s): Lower bound = -69.22932434082031
Epoch 179 (1.7s): Lower bound = -69.29218292236328
Epoch 180 (1.7s): Lower bound = -69.25091552734375
Epoch 181 (1.7s): Lower bound = -69.27210235595703
Epoch 182 (1.7s): Lower bound = -69.25340270996094
Epoch 183 (1.8s): Lower bound = -69.21324157714844
Epoch 184 (1.8s): Lower bound = -69.18798828125
Epoch 185 (1.7s): Lower bound = -69

Epoch 328 (1.8s): Lower bound = -67.95256042480469
Epoch 329 (1.7s): Lower bound = -67.96524810791016
Epoch 330 (1.7s): Lower bound = -67.9251480102539
Epoch 331 (1.8s): Lower bound = -67.9765853881836
Epoch 332 (1.8s): Lower bound = -67.9545669555664
Epoch 333 (1.7s): Lower bound = -67.91631317138672
Epoch 334 (1.7s): Lower bound = -67.95954895019531
Epoch 335 (1.7s): Lower bound = -67.86174011230469
Epoch 336 (1.7s): Lower bound = -67.90972137451172
Epoch 337 (1.7s): Lower bound = -67.96800231933594
Epoch 338 (1.7s): Lower bound = -67.83040618896484
Epoch 339 (1.7s): Lower bound = -67.87809753417969
Epoch 340 (1.7s): Lower bound = -67.84849548339844
Epoch 341 (1.7s): Lower bound = -67.8394775390625
Epoch 342 (1.7s): Lower bound = -67.84442901611328
Epoch 343 (1.7s): Lower bound = -67.85012817382812
Epoch 344 (1.7s): Lower bound = -67.88249206542969
Epoch 345 (1.7s): Lower bound = -67.82015228271484
Epoch 346 (1.7s): Lower bound = -67.95079803466797
Epoch 347 (1.7s): Lower bound = -67

...

Epoch 4315 (1.8s): Lower bound = -65.10185241699219
Epoch 4316 (1.7s): Lower bound = -65.08683776855469
Epoch 4317 (1.7s): Lower bound = -65.07047271728516
Epoch 4318 (1.7s): Lower bound = -65.18303680419922
Epoch 4319 (1.7s): Lower bound = -65.1151351928711
Epoch 4320 (1.8s): Lower bound = -65.02546691894531
Epoch 4321 (1.8s): Lower bound = -65.13191223144531
Epoch 4322 (1.7s): Lower bound = -65.1367416381836
Epoch 4323 (1.8s): Lower bound = -65.13345336914062
Epoch 4324 (1.7s): Lower bound = -65.12993621826172
Epoch 4325 (1.8s): Lower bound = -65.09972381591797
Epoch 4326 (1.7s): Lower bound = -65.13796997070312
Epoch 4327 (1.7s): Lower bound = -65.10348510742188
Epoch 4328 (1.7s): Lower bound = -65.09733581542969
Epoch 4329 (1.7s): Lower bound = -65.09759521484375
Epoch 4330 (1.7s): Lower bound = -65.14850616455078
Epoch 4331 (1.8s): Lower bound = -65.0637435913086
Epoch 4332 (1.8s): Lower bound = -65.1490478515625
Epoch 4333 (1.7s): Lower bound = -65.21393585205078
Epoch 4334 (1.7s

Epoch 4474 (1.7s): Lower bound = -65.0499496459961
Epoch 4475 (1.7s): Lower bound = -65.1051254272461
Epoch 4476 (1.7s): Lower bound = -65.08103942871094
Epoch 4477 (1.7s): Lower bound = -65.06678009033203
Epoch 4478 (1.8s): Lower bound = -65.08399963378906
Epoch 4479 (1.7s): Lower bound = -65.14160919189453
Epoch 4480 (1.7s): Lower bound = -65.08619689941406
Epoch 4481 (1.7s): Lower bound = -65.18260192871094
Epoch 4482 (1.7s): Lower bound = -65.06621551513672
Epoch 4483 (1.7s): Lower bound = -65.13871002197266
Epoch 4484 (1.7s): Lower bound = -65.14922332763672
Epoch 4485 (1.8s): Lower bound = -65.0135726928711
Epoch 4486 (1.8s): Lower bound = -65.10411834716797
Epoch 4487 (1.8s): Lower bound = -65.08546447753906
Epoch 4488 (1.7s): Lower bound = -65.09384155273438
Epoch 4489 (1.7s): Lower bound = -65.08755493164062
Epoch 4490 (1.8s): Lower bound = -65.11195373535156
Epoch 4491 (1.7s): Lower bound = -65.13935852050781
Epoch 4492 (1.9s): Lower bound = -65.13420867919922
Epoch 4493 (1.8

Epoch 4633 (1.7s): Lower bound = -65.03424072265625
Epoch 4634 (1.7s): Lower bound = -65.02519989013672
Epoch 4635 (1.8s): Lower bound = -65.08069610595703
Epoch 4636 (1.7s): Lower bound = -65.0595932006836
Epoch 4637 (1.7s): Lower bound = -65.1188735961914
Epoch 4638 (1.7s): Lower bound = -65.03557586669922
Epoch 4639 (1.7s): Lower bound = -65.04979705810547
Epoch 4640 (1.8s): Lower bound = -65.08865356445312
Epoch 4641 (1.8s): Lower bound = -65.0433578491211
Epoch 4642 (1.7s): Lower bound = -65.0083236694336
Epoch 4643 (1.7s): Lower bound = -64.9754409790039
Epoch 4644 (1.8s): Lower bound = -65.17037200927734
Epoch 4645 (1.7s): Lower bound = -65.01702880859375
Epoch 4646 (1.8s): Lower bound = -65.0499038696289
Epoch 4647 (1.8s): Lower bound = -65.11040496826172
Epoch 4648 (1.7s): Lower bound = -65.09003448486328
Epoch 4649 (1.7s): Lower bound = -65.0757064819336
Epoch 4650 (1.7s): Lower bound = -65.05964660644531
Epoch 4651 (1.8s): Lower bound = -65.06085205078125
Epoch 4652 (1.7s): 

Epoch 4792 (1.7s): Lower bound = -65.064208984375
Epoch 4793 (1.7s): Lower bound = -65.05923461914062
Epoch 4794 (1.7s): Lower bound = -65.13986206054688
Epoch 4795 (1.8s): Lower bound = -65.03709411621094
Epoch 4796 (1.8s): Lower bound = -65.01274871826172
Epoch 4797 (1.7s): Lower bound = -65.03077697753906
Epoch 4798 (1.7s): Lower bound = -65.03244018554688
Epoch 4799 (1.7s): Lower bound = -65.03863525390625
Epoch 4800 (1.8s): Lower bound = -64.98916625976562
Epoch 4801 (1.9s): Lower bound = -65.07540130615234
Epoch 4802 (1.7s): Lower bound = -65.02278900146484
Epoch 4803 (1.8s): Lower bound = -65.02159881591797
Epoch 4804 (1.7s): Lower bound = -65.09796142578125
Epoch 4805 (1.7s): Lower bound = -65.08155822753906
Epoch 4806 (1.7s): Lower bound = -64.97952270507812
Epoch 4807 (1.7s): Lower bound = -64.98417663574219
Epoch 4808 (1.7s): Lower bound = -65.09461975097656
Epoch 4809 (1.7s): Lower bound = -65.0709228515625
Epoch 4810 (1.7s): Lower bound = -65.04679870605469
Epoch 4811 (1.7

Epoch 4951 (1.7s): Lower bound = -64.99433898925781
Epoch 4952 (1.7s): Lower bound = -65.06224060058594
Epoch 4953 (1.7s): Lower bound = -65.0167236328125
Epoch 4954 (1.7s): Lower bound = -64.99836730957031
Epoch 4955 (1.7s): Lower bound = -65.03035736083984
Epoch 4956 (1.7s): Lower bound = -64.9969482421875
Epoch 4957 (1.7s): Lower bound = -65.03490447998047
Epoch 4958 (1.7s): Lower bound = -65.01728820800781
Epoch 4959 (1.8s): Lower bound = -64.96724700927734
Epoch 4960 (1.7s): Lower bound = -65.00334167480469
Epoch 4961 (1.7s): Lower bound = -65.06977081298828
Epoch 4962 (1.7s): Lower bound = -65.07598876953125
Epoch 4963 (1.7s): Lower bound = -65.04955291748047
Epoch 4964 (1.7s): Lower bound = -64.98265838623047
Epoch 4965 (1.7s): Lower bound = -65.0483627319336
Epoch 4966 (1.7s): Lower bound = -65.01676940917969
Epoch 4967 (1.8s): Lower bound = -65.03863525390625
Epoch 4968 (1.8s): Lower bound = -65.02920532226562
Epoch 4969 (1.7s): Lower bound = -64.98699951171875
Epoch 4970 (1.7